In [1]:
!tree ../dataset

../dataset
├── en_espanol
│   ├── docx2txt.py
│   ├── Второй_жанр_исходная.txt
│   └── Первый_жанр_исходная.txt
├── Второй_жанр_исходная.txt
├── Первый_жанр_исходная.txt
├── Сокращение по частям речи
│   ├── 1.Первый жанр исходная выборка.txt
│   ├── 2.Первый жанр без клауз, включающих наречия.txt
│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
│   ├── 4.Первый жанр без клауз, включающих глаголы и наречия.txt
│   ├── 5.без клауз, включающих местоимения.txt
│   ├── 6.без слов функциональных.txt
│   ├── Без прилагательных второй жанр.txt
│   ├── Без прилагательных первый жанр.txt
│   ├── Второй_жанр без клауз, включающих местоимения.txt
│   ├── Второй_жанр без слов функциональных.txt
│   └── Случайные выборки.txt
└── сокращение по частотности
    ├── 1а_ без сокращений.txt
    ├── 1б_Изъяты лексемы с частотой выше 100.txt
    ├── 1в_Изъяты лексемы с частотой выше 49.txt
    ├── 1г_Изъяты лексемы с частотой выше 29.txt
    ├── 1д_Изъяты лексемы с частотой выше 9.txt
    ├── 1е_Изъ

In [2]:
import json

def save_results(results, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=4, ensure_ascii=False)

In [3]:
import funciones
from utils import train_wrapper
import warnings
import os
import torch
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from contextlib import redirect_stderr
import multiprocessing as mp
import numpy as np
#import wandb
import nbformat
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sklearn.metrics import classification_report, confusion_matrix
import gc
import re
import nltk
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, AdamW
from razdel import sentenize
import numpy as np

# Suprimir warnings específicos
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
mp.set_start_method('spawn', force=True)


import json


def convert_numpy(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, (np.float32, np.float64, np.int32, np.int64)):
        return obj.item()
    return obj

def save_results(results, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=4, ensure_ascii=False, default=convert_numpy)

# import funciones
# from utils import train_wrapper
# import warnings
# import os
# import torch
# import matplotlib.pyplot as plt

# # Suprimir warnings específicos
# warnings.filterwarnings('ignore', category=UserWarning)  # Para sklearn y otros
# warnings.filterwarnings('ignore', category=FutureWarning)  # Para huggingface y transformers
# from concurrent.futures import ProcessPoolExecutor
# from tqdm import tqdm
# from contextlib import redirect_stderr
# from concurrent.futures import ProcessPoolExecutor
# import multiprocessing as mp
# import numpy as np
# mp.set_start_method('spawn', force=True)
# #os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

###############################################################################
# import wandb
# import nbformat
# # Configurar la clave de la API como variable de entorno
# os.environ["WANDB_API_KEY"] = "ca85316ed713b2425615fb3a613d7eb414c9f57f"  # Reemplaza con tu clave
# # Iniciar wandb sin especificar entity (se detecta automáticamente)
# wandb.init(project="model-clasification")
# wandb.init(
#     settings=wandb.Settings(
#         start_method="thread",
#         timeout=30,
#         sync_dir="/tmp/wandb",  # Usa un directorio temporal
#         disable_code=True       # Mejora la estabilidad
#     )
# )
#os.environ["WANDB_NOTEBOOK_NAME"] = "сокращение по частотности.ipynb"
###############################################################################


def create_custom_config(model_name, model_type, dataset):
    """Crea configuración de entrenamiento adaptativa"""
    common_config = {
#         'dataset_name':dataset.get('name', ''),
        'model_name': model_name,
        'model_type': model_type,
        'num_repeats': 6,
        'test_size': 0.2,
        'threshold': 0.5
    }

    # Configuración específica para GPT
    if model_type == 'gpt':
        return funciones.TrainingConfig(
            **common_config,
            max_length=52,
            batch_size=64,
            epochs=6,
            learning_rate=1e-5
        )
#         freq_threshold = dataset.get('freq_threshold')
#         if freq_threshold in [100, 49, 29, 9, 5, 3]:
#             configs = {
#                 100: (52, 32, 6, 1e-5),#test 1
#                 49: (60, 128, 4, 3e-5),
#                 29: (51, 128, 4, 3e-5),
#                 9: (45, 128, 5, 4e-5),
#                 5: (150, 32, 6, 5e-5),
#                 3: (150, 32, 6, 5e-5)
#             }
#             max_len, batch, epochs, lr = configs[freq_threshold]
#             return funciones.TrainingConfig(
#                 **common_config,
#                 max_length=max_len,
#                 batch_size=batch,
#                 epochs=epochs,
#                 learning_rate=lr
#             )

        # Configuración por nombre de dataset
        dataset_name = dataset.get('name', '')
        if any(x in dataset_name for x in ['1', '2', '3', '4']):
            return funciones.TrainingConfig(
                **common_config,
                max_length=60,
                batch_size=128,
                epochs=6,
                learning_rate=5e-5
            )


    # Configuraciones basadas en frecuencia
    freq_threshold = dataset.get('freq_threshold')
    if freq_threshold in [100, 49, 29, 9, 5, 3]:
        configs = {
            100: (52, 128, 3, 2e-5),
            49: (60, 128, 4, 3e-5),
            29: (51, 128, 4, 3e-5),
            9: (45, 128, 5, 4e-5),
            5: (150, 32, 6, 5e-5),
            3: (150, 32, 6, 5e-5)
        }
        max_len, batch, epochs, lr = configs[freq_threshold]
        return funciones.TrainingConfig(
            **common_config,
            max_length=max_len,
            batch_size=batch,
            epochs=epochs,
            learning_rate=lr
        )

    # Configuración por nombre de dataset
    dataset_name = dataset.get('name', '')
    if any(x in dataset_name for x in ['1', '2', '3', '4','5','6']):
        configs = {
            1: (60, 128, 3, 5e-5),
            2: (60, 128, 4, 5e-5),
            3: (60, 128, 4, 5e-5),
            4: (60, 128, 5, 5e-5),
            5: (60, 128, 6, 5e-5),
            6: (60, 128, 6, 5e-5)
        }
        max_len, batch, epochs, lr = configs[x]
        
        return funciones.TrainingConfig(
            **common_config,
            max_length=max_len,
            batch_size=batch,
            epochs=epochs,
            learning_rate=lr
        )

    # Configuración por defecto
    return funciones.TrainingConfig(
        **common_config,
        max_length=60,
        batch_size=32,
        epochs=4,
        learning_rate=3e-5
    )

models = [
        {'model':'DeepPavlov/rubert-base-cased',
         'name':'DeepPavlov-rubert-base',
         'type': 'bert'},# Modelo original
        {'model':'bert-base-multilingual-cased',
         'name':'BERT multilingual',
         'type': 'bert'},# BERT multilingüe
        {'model':'distilbert-base-multilingual-cased',
         'name':'distilbert-base-multilingual',
         'type': 'bert'},# Versión ligera de BERT
        {'model':'roberta-base',
         'name':'roberta-base', 
        'type': 'bert'}, # RoBERTa 
        {'model': 'gpt2',
         'name': 'gpt2',
         'type': 'gpt'},  #  GPT model
        {'model': 'facebook/opt-125m',
         'name': 'facebook',
         'type': 'gpt'},  #  GPT model
        {'model': 'sberbank-ai/rugpt3small_based_on_gpt2',
         'name': 'rugpt3',
         'type': 'gpt'}
    ]

datasets = [
        {
            'path1': '../dataset/сокращение по частотности/1б_Изъяты лексемы с частотой выше 100.txt',
            'path2': '../dataset/сокращение по частотности/2б_Изъяты лексемы с частотой выше 100.txt',
            'name': 'Изъяты лексемы с частотой выше 100',
            'type': 'freq',
            'freq': 100
        },
        {
            'path1': '../dataset/сокращение по частотности/1в_Изъяты лексемы с частотой выше 49.txt',
            'path2': '../dataset/сокращение по частотности/2в_Изъяты лексемы с частотой выше 49.txt',
            'name': 'Изъяты лексемы с частотой выше 49',
            'type': 'freq',
            'freq': 49
        },
        {
            'path1': '../dataset/сокращение по частотности/1г_Изъяты лексемы с частотой выше 29.txt',
            'path2': '../dataset/сокращение по частотности/2г_Изъяты лексемы с частотой выше 29.txt',
            'name': 'Изъяты лексемы с частотой выше 29',
            'type': 'freq',
            'freq': 29
        },
        {
            'path1': '../dataset/сокращение по частотности/1д_Изъяты лексемы с частотой выше 9.txt',
            'path2': '../dataset/сокращение по частотности/2д_Изъяты лексемы с частотой выше 9.txt',
            'name': 'Изъяты лексемы с частотой выше 9',
            'type': 'freq',
            'freq': 9
        },
        {
            'path1': '../dataset/сокращение по частотности/1е_Изъяты лексемы с частотой выше 5.txt',
            'path2': '../dataset/сокращение по частотности/2е_Изъяты лексемы с частотой выше 5.txt',
            'name': 'Изъяты лексемы с частотой выше 5',
            'type': 'freq',
            'freq': 5
        },
        {
            'path1': '../dataset/сокращение по частотности/1ё_Изъяты лексемы с частотой выше 3.txt',
            'path2': '../dataset/сокращение по частотности/2ё_Изъяты лексемы с частотой выше 3.txt',
            'name': 'Изъяты лексемы с частотой выше 3',
            'type': 'freq',
            'freq': 3
        },

            {
            'path1': '../dataset/Сокращение по частям речи/Без прилагательных первый жанр.txt',
            'path2': '../dataset/Сокращение по частям речи/Без прилагательных второй жанр.txt',
            'name': 'Без прилагательных первый-второй жанр',#litle correction
            'type': 'pos',
            'freq': None
            },
          {
            'path1': '../dataset/Сокращение по частям речи/1.Первый жанр исходная выборка.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '1.Первый жанр исходная выборка',
            'type': 'pos',
            'freq': None

        },
        {
            'path1': '../dataset/Сокращение по частям речи/2.Первый жанр без клауз, включающих наречия.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '2.Первый жанр без клауз, включающих наречия',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/3.Первый жанр без клауз, включающих глаголы.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '3.Первый жанр без клауз, включающих глаголы',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/4.Первый жанр без клауз, включающих глаголы и наречия.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '4.Первый жанр без клауз, включающих глаголы и наречия',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/5.без клауз, включающих местоимения.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '5.без клауз, включающих местоимения.txt',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/6.без слов функциональных.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '6.без слов функциональных.txt',
            'type': 'pos',
            'freq': None
        },
    
    ]



russian_templates = [
    "Этот литературный фрагмент принадлежит к жанру {}.",
    "Данный текст является примером жанра {}.",
    "Стилистические особенности указывают на жанр {}.",
    "Жанровая принадлежность этого текста - {}.",
    "Эксперты классифицируют этот текст как жанр {}.",
    "Это типичный пример жанра {} в русской литературе.",
    "По ключевым характеристикам это текст жанра {}.",
    "Данное произведение относится к направлению {}.",
    "Анализ содержания позволяет отнести текст к жанру {}.",
    "Этот отрывок характерен для жанра {}."
]

def main():

    results = []
    for model in models:
        for dataset in datasets:
            config = create_custom_config(model['model'], model['type'], dataset)
            
            
            # Entrenar y limpiar memoria
            torch.cuda.empty_cache()
            result = funciones.train_and_evaluate_dataset(
                dataset['path1'],
                dataset['path2'],
                config,
                dataset['name'],
                dataset['type']
                
            )
            results.append(result) 
#             if model['type'] = 'gpt':
                
        
        del model
        torch.cuda.empty_cache()
        gc.collect()
        
    
    save_results(results, 'resultados_Step_8.json') 
    
    
    

    
if __name__ == "__main__":
    main()

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
#batch_size=128

In [ ]:
#display_results()

In [ ]:
import os
import IPython

# Reiniciar el kernel
IPython.display.display(IPython.display.Javascript("Jupyter.notebook.kernel.restart()"))

# Apagar el kernel después de reiniciar
os._exit(0)